# TP Optimisation des calculs
But : Optimiser les calculs afin que le PoC soit exécutable sur l'échantillon des modules de production (~100 000 modules de taille 1024 bits) en un temps raisonnable.

Il existe plusieurs manières d'améliorer et optimiser l'exécution d'un programme, ici nous verrons :
- L'optimisation algorithmique
- L'optimisation via des librairies dédiées aux calculs rapides
- L'optimisation via la parallélisation des tâches

## Génération d'un échantillon de test
Si besoin, voici le code afin de générer un échantillon de plusieurs modules dont certains ont un facteur en commun.

In [ ]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import rsa
from collections import defaultdict

from math import ceil

In [ ]:
# Génération d'un biclef -> (n, p, q)
def e_v_rsa_key_pair(modulus_len):
    pri_key = rsa.generate_private_key(public_exponent = 3, key_size = modulus_len, backend = default_backend())
    pub_num = pri_key.public_key().public_numbers()
    pri_num = pri_key.private_numbers()
    return pub_num.n, pri_num.p, pri_num.q


In [ ]:
# Génération d'un jeu de test —— avec quelques cas tordus
# >> Module « calimero » : ses deux facteurs premiers se retrouvent dans le super-module (c'est vraiment trop inzuste)
# >> Modules « super-calimeros » : calimeros qui se calimérisent entre eux
# >> Doublons
def e_v_rsa_test_vector(modulus_len, modulus_count, filename_moduli):
    print("Génération des biclefs de test...")
    percent = ceil(modulus_count/100)

    key_pairs = []

    # n1 = p1 * q1 -> Cassé avec c
    # n2 = p2 * q2 -> Cassé avec c
    #  c = p1 * p2 -> Calimero

    kp1 = e_v_rsa_key_pair(modulus_len)
    kp2 = e_v_rsa_key_pair(modulus_len)

    p1 = kp1[1]
    p2 = kp2[1]

    c = (p1 * p2, p1, p2)

    key_pairs.append(kp1)
    key_pairs.append(kp2)

    # s1 = x * y |
    # s2 = y * z | -> Super-calimeros (ou cyclo-calimeros)
    # s3 = z * x |

    s1 = e_v_rsa_key_pair(modulus_len)
    s2 = e_v_rsa_key_pair(modulus_len)

    x = s1[1]
    y = s1[2]
    z = s2[2]

    s2 = y * z, y, z
    s3 = z * x, z, x

    key_pairs.append(s1)
    key_pairs.append(s2)

    for i in range(modulus_count - 3 - 3):

        if i % percent == 0: print(f"{i // percent}% ", end = "")

        key_pairs.append(e_v_rsa_key_pair(modulus_len))

    print("\n")

    key_pairs.append(c)
    key_pairs.append(c) # Doublon (hyper-calimero)
    key_pairs.append(s3)
    key_pairs.append(s3)
    key_pairs.append(s3)

    with open(filename_moduli, 'w') as f: f.writelines( [ f"{kp[0]:x}\n" for kp in key_pairs ] )


In [ ]:
# Vérification du fichier des modules, suppression des doublons
def e_vs_rsa_check(filename_moduli):
    with open(filename_moduli, 'r') as f: moduli = [ line.strip() for line in f ]
    moduli_dic = defaultdict(list)
    for i, modulus in enumerate(moduli, 1): moduli_dic[modulus].append(i)
    duplicates = [ (indices, modulus) for modulus, indices in moduli_dic.items() if len(indices) > 1 ]
    print(f"Modules dupliqués... {len(duplicates)} !\n")

    if duplicates != []:
        for indices, modulus in duplicates: print( f"# {indices}\nn = {modulus}\n" )
        with open(filename_moduli, 'w') as f: f.writelines( [ modulus + "\n" for modulus, _ in moduli_dic.items() ] )


In [ ]:
# Paramètres pour la génération de modules
modulus_len = 1024
#modulus_count = 1_000
#modulus_count = 10_000
modulus_count = 100_000

filename_moduli = f"e_vs_rsa_moduli_{modulus_len}b×{modulus_count}.txt"

# Génération des modules
e_v_rsa_test_vector(modulus_len, modulus_count, filename_moduli)

# Vérification si doublons
e_vs_rsa_check(filename_moduli)

print(f"Modules enregistrés dans le fichier : {filename_moduli}")

## Code sans optimisation

In [ ]:
from math import ceil, gcd

In [ ]:
# Coeur du PoC, vérification des facteurs communs
def e_vs_rsa_qad(moduli):
    euclided = []
    percent = ceil( len(moduli) / 100 )
    compteur = 0
    
    # Parcours simple, sinon simpliste, des modules entre eux
    for i, modulus1 in enumerate(moduli, 1):
        
        # Affichage de l'avancement
        if i % percent == 0: 
            print(f"{i // percent}% ", end = "")

        for j, modulus2 in enumerate(moduli[i:], i+1):
            compteur += 1
            # Verification si module en commun
            g = gcd(modulus1, modulus2)
            
            if g != 1: 
                euclided.append( (i, j, modulus1, g, modulus1 // g) )
                euclided.append( (j, i, modulus2, g, modulus2 // g) )

    print(f"\n Nombre d'itérations : {compteur}")
    return euclided

In [ ]:
# Fonction principale
def e_vs_rsa(filename_moduli):
    # Récupération des modules stockés dans un fichier
    with open(filename_moduli, "r") as f_moduli:
        moduli = [ int(line, base=16) for line in f_moduli ]

    # Vérification des facteurs communs
    euclided = e_vs_rsa_qad(moduli)
    
    # Ecriture et affichage des résultats
    with open(filename_moduli + ".txt", 'w') as f:
        res = f"Modules factorisés... {len(euclided)} !\n\n"

        for i, j, n, p, q in euclided: res += f"# {i} (avec {j})\nn = {n:#x} \np = {p:#x} \nq = {q:#x}\n\n"

        res = res.rstrip()

        f.writelines(res)

        print("\n\n" + res)

In [ ]:
%%time
filename_moduli = "e_vs_rsa_mod_1024_10_3.txt"
#filename_moduli = "e_vs_rsa_mod_1024_10_4.txt"
#filename_moduli = "e_vs_rsa_mod_1024_10_5.txt"

# Recherche de facteurs communs
e_vs_rsa(filename_moduli)

## Optimisation algorithmique : le super-module

In [ ]:
from math import ceil, gcd

Pour calculer le super-module il est nécessaire de multiplier tous les modules entre eux (ici stockés dans une liste). Quelle est la manière naïve de multiplier tous les éléments d'une liste ?

In [ ]:
# Calcul du grand produit à partir d'une liste de facteurs - A compléter
def mult(factors):
    # A compléter, doit retourner le produit d'une liste


<details>

<summary>Solution</summary>

```python
def mult(factors):
    res = 1

    for factor in factors:
        res*= factor
    
    return res
```
</details>

In [ ]:
# Calcul du super-module et écriture de ce dernier —— plus subtil qu'il n'y paraît
def e_vs_rsa_super(filename_moduli):
    # Récupération des modules stockés dans un fichier
    with open(filename_moduli, "r") as f: 
        moduli = [ int(line, base=16) for line in f ]
    
    # Calcul du super-module
    super_modulus = mult(moduli)
    
    # Ecriture du super-module
    with open(filename_moduli + ".bin", "wb") as f:
        super_modulus_len = ceil(super_modulus.bit_length()/8)
        f.write(super_modulus.to_bytes(super_modulus_len, byteorder="big"))

In [ ]:
# Coeur du PoC, vérification des facteurs communs
def e_vs_rsa_qad(moduli, super_modulus):
    euclided, usual_suspects = [], []
    percent = ceil( len(moduli) / 100 )

    print("Parcours super-module...")

    # Parcours simple, sinon simpliste, des modules contre le super-module
    for i, modulus in enumerate(moduli, 1):
        
        # Affichage de l'avancement
        if i % percent == 0: 
            print(f"{i // percent}% ", end = "")

        # Verification si module en commun
        g = gcd(modulus, super_modulus // modulus)

        if g != 1: 
            usual_suspects.append( (i, modulus) ) # On en tient un

    # Parcours quadratique, réputé bref, des modules factorisables
    for i, modulus in usual_suspects:
        for j, not_modulus in usual_suspects:

            g = gcd(modulus, not_modulus)

            if 1 < g < modulus:

                euclided.append( (i, j, modulus, g, modulus // g) )

                break

    return euclided

In [ ]:
# Fonction principale
def e_vs_rsa(filename_moduli):
    # Récupération des modules stockés dans un fichier
    with open(filename_moduli, "r") as f_moduli, open(filename_moduli + ".bin", "rb") as f_super_modulus:
        moduli = [ int(line, base=16) for line in f_moduli ]
        
        # Récupération du super-module
        super_modulus = int.from_bytes(f_super_modulus.read(), byteorder="big")

    # Vérification des facteurs communs
    euclided = e_vs_rsa_qad(moduli, super_modulus)

    # Ecriture et affichage des résultats
    with open(filename_moduli + ".txt", 'w') as f:
        res = f"Modules factorisés... {len(euclided)} !\n\n"

        for i, j, n, p, q in euclided: res += f"# {i} (avec {j})\nn = {n:#x} \np = {p:#x} \nq = {q:#x}\n\n"

        res = res.rstrip()

        f.writelines(res)

        print("\n\n" + res)

In [ ]:
%%time
filename_moduli = "e_vs_rsa_mod_1024_10_3.txt"
#filename_moduli = "e_vs_rsa_mod_1024_10_4.txt"
#filename_moduli = "e_vs_rsa_mod_1024_10_5.txt"

# Calcul du super modulo
e_vs_rsa_super(filename_moduli)

In [ ]:
%%time
# Recherche de facteurs communs
e_vs_rsa(filename_moduli)

## Optimisation algorithmique : la multiplication

In [ ]:
# Calcul du grand produit optimisé de manière récursive - A compléter
def mult(factors):
    # A compléter, doit retourner le produit d'une liste
    
    

<details>

<summary>Solution</summary>

```python
def mult(factors):
    factors_len = len(factors)
    
    if factors_len == 1: 
        return factors[0]
    
    else: 
        return mult( factors[:factors_len//2] ) * mult( factors[factors_len//2:] )
```
</details>

In [ ]:
%%time
filename_moduli = "e_vs_rsa_mod_1024_10_3.txt"
#filename_moduli = "e_vs_rsa_mod_1024_10_4.txt"
#filename_moduli = "e_vs_rsa_mod_1024_10_5.txt"

# Calcul du super modulo
e_vs_rsa_super(filename_moduli)

## Optimisation via l'utilisation de librairies dédiées aux calculs rapides
Ici via l'utilisation de la librairie gmpy2 (https://gmpy2.readthedocs.io/en/latest/)

A noter que l'installation d'une librairie est importante, les librairies proposées par Anaconda sont par exemple plus optimisées (dont Numpy, Pandas, Scikit, ...).

In [ ]:
from math import ceil

from gmpy2 import mpz, mul, to_binary, from_binary, gcd, divexact

In [ ]:
# Calcul du grand produit —— plus subtil qu'il n'y paraît
def mult(factors):
    factors_len = len(factors)
    
    if factors_len == 1: 
        return factors[0]
    
    else: 
        return mul(mult( factors[:factors_len//2] ), mult( factors[factors_len//2:] ))

In [ ]:
# Calcul du super-module et écriture binaire de ce dernier
def e_vs_rsa_super(filename_moduli):
    with open(filename_moduli, 'r') as f:
        moduli = [ mpz(int(line, base=16)) for line in f ] # Il y a du changement ici
        super_modulus = mult(moduli)
        
    with open(filename_moduli + ".bin", "wb") as f : # Il y a du changement ici
        f.write(to_binary(super_modulus))

In [ ]:
# Coeur du PoC, vérification des facteurs communs
def e_vs_rsa_qad(moduli, super_modulus):
    euclided, usual_suspects = [], []
    percent = ceil( len(moduli) / 100 )

    print("Parcours super-module...")

    # Parcours simple, sinon simpliste, des modules contre le super-module
    for i, modulus in enumerate(moduli, 1):

        if i % percent == 0: 
            print(f"{i // percent}% ", end = "")

        g = gcd(modulus, divexact(super_modulus, modulus))

        if g != 1: 
            usual_suspects.append( (i, modulus) ) # On en tient un

    # Parcours quadratique, réputé bref, des modules factorisables
    for i, modulus in usual_suspects:
        for j, not_modulus in usual_suspects:

            g = gcd(modulus, not_modulus)

            if 1 < g < modulus:

                euclided.append( (i, j, modulus, g, divexact(modulus, g)) )

                break

    return euclided

In [ ]:
# Fonction principale
def e_vs_rsa(filename_moduli):

    with open(filename_moduli, "r") as f_moduli, open(filename_moduli + ".bin", "rb") as f_super_modulus:
        moduli = [ mpz(int(line, base=16)) for line in f_moduli ] # Il y a du changement ici
        super_modulus = from_binary(f_super_modulus.read())

    euclided = e_vs_rsa_qad(moduli, super_modulus)

    with open(filename_moduli + ".txt", 'w') as f:

        res = f"Modules factorisés... {len(euclided)} !\n\n"

        for i, j, n, p, q in euclided: res += f"# {i} (avec {j})\nn = {n:#x} \np = {p:#x} \nq = {q:#x}\n\n"

        res = res.rstrip()

        f.writelines(res)

        print("\n\n" + res)

In [ ]:
%%time
filename_moduli = "e_vs_rsa_mod_1024_10_3.txt"
#filename_moduli = "e_vs_rsa_mod_1024_10_4.txt"
#filename_moduli = "e_vs_rsa_mod_1024_10_5.txt"

# Calcul du super modulo
e_vs_rsa_super(filename_moduli)

In [ ]:
%%time
# Recherche de facteurs communs
e_vs_rsa(filename_moduli)

## Optimisation via parallélisation des calculs
Ici via l'utilisation de la librairie multiprocessing (https://docs.python.org/3/library/multiprocessing.html)

In [ ]:
# Commande pour déterminer le nombre de cores CPU disponibles sur le système
!lscpu

In [ ]:
from gmpy2 import mpz, mul, to_binary, from_binary, gcd, divexact

from multiprocessing import Pool
import numpy as np

In [ ]:
# Vérification si module premier avec le supermodule, fonction appelée par le pool de process
def compute_modulus(modulus, super_modulus):
    # A compléter, doit retourner une liste de modules
    
    

<details>

<summary>Solution</summary>

```python
def compute_modulus(modulus, super_modulus):
    suspects = []
    for m in modulus :
        g = gcd(m, divexact(super_modulus, m))
        if g != 1:
            suspects.append(m)
    return suspects
```
</details>

In [ ]:
# Fonction (callback) appelée à la fin d'un processus, permet de récupérer les résultats
def compute_result(suspects):
    global usual_suspects
    usual_suspects += suspects

In [ ]:
# Coeur du PoC, vérification des facteurs communs
def e_vs_rsa_qad(moduli, super_modulus, process_number):
    euclided = []

    print("Parcours super-module...")

    # Création du pool de process
    pool = Pool(processes=process_number)

    # On coupe la liste de modules en n sous listes avec n le nombre de processus
    split_moduli = np.array_split(moduli, process_number)

    # On parcourt les différentes sous-listes, puis on ajoute au pool les arguments
    for modulus in split_moduli :
        pool.apply_async(compute_modulus, args=(modulus, super_modulus), callback=compute_result)
    
    # On n'a plus besoin de la pool donc on peut la fermer
    pool.close()

    # On attend le travail des workers
    pool.join()
    
  # Parcours quadratique, réputé bref, des modules factorisables
    for modulus in usual_suspects:
        for not_modulus in usual_suspects:
            g = gcd(modulus, not_modulus)
            if 1 < g < modulus:
                euclided.append( (modulus, g, divexact(modulus, g)) )
                break

    return euclided

In [ ]:
# Fonction principale
def e_vs_rsa(filename_moduli, process_number):
    with open(filename_moduli, "r") as f_moduli, open(filename_moduli + ".bin", "rb") as f_super_modulus:
        moduli = [ mpz(int(line, base=16)) for line in f_moduli ]
        super_modulus = from_binary(f_super_modulus.read())

    euclided = e_vs_rsa_qad(moduli, super_modulus, process_number)

    with open(filename_moduli + ".txt", 'w') as f:
        res = f"Modules factorisés... {len(euclided)} !\n\n"
        for n, p, q in euclided: res += f"n = {n:#x} \np = {p:#x} \nq = {q:#x}\n\n"
        res = res.rstrip()
        f.writelines(res)
        print("\n\n" + res)

In [ ]:
%%time
filename_moduli = "e_vs_rsa_mod_1024_10_3.txt"
#filename_moduli = "e_vs_rsa_mod_1024_10_4.txt"
#filename_moduli = "e_vs_rsa_mod_1024_10_5.txt"

# Calcul du super modulo
e_vs_rsa_super(filename_moduli)

In [ ]:
%%time
# Nombre de processus à utiliser, doit être inférieur ou égal au nombre de cores
process_number = 4

# On initialise la variable globale qui sera modifiée par les callbacks
usual_suspects = []

# Recherche de facteurs communs
e_vs_rsa(filename_moduli, process_number)